# Requesting Comments

1. Insert your API Key from https://console.cloud.google.com/apis/api/youtube.googleapis.com to 'DEVELOPER_KEY' in your '.env' as a string.
2. Insert your video id to 'videoId' as string. Example: https://www.youtube.com/watch?v=SIm2W9TtzR0, videoId='SIm2W9TtzR0'

**Example**

In [ ]:
import googleapiclient.discovery
import googleapiclient.errors

api_service_name = 'youtube'
api_version = 'v3'

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

request = youtube.commentThreads().list(
    part='snippet',
    videoId='0FtcHjI5lmw',
    maxResults=100
)
response = request.execute()
response

# for item in response['items']:
#     print(item['snippet']['topLevelComment']['snippet']['textDisplay'])

# Requesting Comments to DataFrame

**Import DEVELOPER_KEY, IMPORTANT!**<br>
**RUN THIS FIRST BEFORE ANYTHING ELSE**

In [4]:
import os
from dotenv import load_dotenv

load_dotenv()
DEVELOPER_KEY = os.getenv('DEVELOPER_KEY')

### Request for specific channel

**Imports & Youtube API Init**

In [5]:
import googleapiclient.discovery
import googleapiclient.errors
import pandas as pd

api_service_name = 'youtube'
api_version = 'v3'

youtube = googleapiclient.discovery.build(api_service_name, api_version, developerKey=DEVELOPER_KEY)

**Request Video Ids using Search**

In [47]:
CHANNEL_ID = 'UCaizTs-t-jXjj8H0-S3ATYA'

request_video_ids = youtube.search().list(
    part='snippet',
    channelId=CHANNEL_ID,
    maxResults=50,
    type='video',
)
response_video_ids = request_video_ids.execute()

video_ids = []
    
while True:
    for item in response_video_ids['items']:
        video_ids.append(item['id']['videoId'])
    try:
        next_page_token_search = response_video_ids['nextPageToken']
    except KeyError:
        break
    request_video_ids = youtube.search().list(
        part='snippet',
        channelId=CHANNEL_ID,
        maxResults=50,
        type='video',
        pageToken=next_page_token_search
    )
    response_video_ids = request_video_ids.execute()

**Request Comments for every Video Id**

In [ ]:
comments = []

for video_id in video_ids:
    request_comments = youtube.commentThreads().list(
        part='id,replies,snippet',
        videoId=video_id,
        maxResults=1000
    )
    response_comments = request_comments.execute()

    while True:
        for item in response_comments['items']:
            comment = item['snippet']['topLevelComment']['snippet']
            comments.append([
            comment['authorDisplayName'],
            comment['publishedAt'],
            comment['likeCount'],
            comment['textOriginal']
        ])
        try:
            next_page_token_comments = response_comments['nextPageToken']
        except KeyError:
            break
        request_comments = youtube.commentThreads().list(
            part='id,replies,snippet',
            videoId=video_id,
            maxResults=1000,
            pageToken=next_page_token_comments
        )
        response_comments = request_comments.execute()

df = pd.DataFrame(comments, columns=['author', 'published_at', 'like_count', 'text'])

In [ ]:
# df.head()
df.sort_values('like_count', ascending=False)